In [8]:
import pdfplumber

def extract_text_from_pdf(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        text = ''
        for page in pdf.pages:
            text += page.extract_text()
    return text

# Replace 'your_pdf_file.pdf' with the actual path to your PDF file
pdf_path = r'C:\Users\salom\OneDrive\Área de Trabalho\20.4pdf.pdf'
extracted_text = extract_text_from_pdf(pdf_path)


In [35]:
with open('20.4_text.txt', 'w', encoding='utf-8') as file:
    file.write(extracted_text)

In [6]:
import re

def extract_isbns(text):
    # Define a regex pattern to match strings of numbers with dashes
    isbn_pattern = r'\b\d{1,5}-?\d{1,7}-?\d{1,7}-?\d{1,7}-?\w\b'

    # Find all matches of the ISBN pattern in the text
    isbns = re.findall(isbn_pattern, text)

    return isbns

# Extract ISBNs from the example text
isbns_found = extract_isbns(extracted_text)

# Print the list of ISBNs
print("ISBNs found:", isbns_found)

ISBNs found: ['11-01-1997', '28036382', '0741-6555', '94110', '882554', '94188-2554', '415-642-9993', '415-642-9995', '882554', '94188', '94110', '77098', '404-815', '04104', '3005-4', '3004-6', '2899-8', '2898-X', '2439-9', '2441-0', '2440-2', '425-815', '604-687-2919', '617-354-5201', '94621', '999-4650', '632-1281', '90048', '213-651-0880', '95061', '408-426-9172', '0-87286-332-8', '29681', '864-862-7833', '864-862-6944', '615-793-5555', '20036', '800-969-6676', '617-262-6969', '941-922-0844', '1-880913-16-X', '1-55074-113-6', '0-670-87367-5', '0-452-27621-7', '1-55164-058-9', '0-15-100263-0', '0-520-20596-0', '882554', '0-679-45731-3', '92663', '714-631-1179', '94523', '510-947-5571', '510-947-5579', '306-653-7966', '306-665-0276', '65806', '417-887-4521', '85716', '520-318-4307', '520-319-8496', '93728', '209-233-3600', '90265', '310-317-9573', '310-456-5774', '65161', '85739', '648-5333', '43215', '614-291-0080', '614-291-0081', '250-361-9718', '250-361-9738', '0-9644196-4-5', '1

In [12]:
import pdfplumber
import pandas as pd
import re

def extract_isbns_from_text(text):
    # Define the ISBN regex pattern
    isbn_pattern = re.compile(r'\b\d{1,5}-?\d{1,7}-?\d{1,7}-?\d{1,7}-?\w\b')

    # Find all ISBNs in the text
    isbns_found = isbn_pattern.findall(text)
    return isbns_found

def extract_text_from_pdf(pdf_path):
    data = {'PageNumber': [], 'Text': [], 'ISBNs': []}
    with pdfplumber.open(pdf_path) as pdf:
        for i, page in enumerate(pdf.pages, start=1):
            # Add page number and corresponding text to the data dictionary
            data['PageNumber'].append(i)
            text = page.extract_text()
            data['Text'].append(text)
            
            # Extract ISBNs from the text
            isbns_found = extract_isbns_from_text(text)
            data['ISBNs'].append(isbns_found)

    return pd.DataFrame(data)

# Replace 'your_pdf_file.pdf' with the actual path to your PDF file
pdf_path = r'C:\Users\salom\OneDrive\Área de Trabalho\20.4pdf.pdf'
df = extract_text_from_pdf(pdf_path)


In [13]:
df.head()

,PageNumber,Text,ISBNs
0,1,Feminist Bookstore News\nSource: Reveal Digita...,"[11-01-1997, 28036382]"
1,2,Feminist\nBookstore\nNews EINE\nVolume 20 Numb...,[]
2,3,AUASTE a aA SA Authors\nTHE SECRET LANGUAGE OF...,[]
3,4,Feminist Bookstore News\nAs Wwe go to press......,[]
4,5,Feminist Bookstore News\nyou think — printable...,"[0741-6555, 94110, 882554, 94188-2554]"


In [34]:
df.to_csv('20.4_text.csv', index=False)

In [14]:
import requests
import csv

api_key = '51003_209599d3945c0e0c1aab5978b2fa0df3'
isbns_to_process = isbns_found  # Replace with the original list of ISBNs you want to process

# Construct the base URL
base_url = 'https://api2.isbndb.com/book/'

# Create a CSV file and write headers
with open('book_information.csv', 'w', newline='', encoding='utf-8') as csv_file:
    fieldnames = ['ISBN', 'Title', 'Author', 'Publisher', 'Pages', 'Date Published', 'Binding', 'Synopsis', 'Language', 'Edition', 'Dimensions', 'MSRP', 'Image', 'Status']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

    writer.writeheader()

    # Make requests for each ISBN in the list
    for isbn in isbns_to_process:
        url = f'{base_url}{isbn}'
        headers = {'Authorization': api_key}

        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            book_info = response.json().get('book', {})
            
            # Write information to CSV
            writer.writerow({
                'ISBN': isbn,
                'Title': book_info.get('title', ''),
                'Author': ', '.join(book_info.get('authors', [])),
                'Publisher': book_info.get('publisher', ''),
                'Pages': book_info.get('pages', ''),
                'Date Published': book_info.get('date_published', ''),
                'Binding': book_info.get('binding', ''),
                'Synopsis': book_info.get('synopsis', ''),
                'Language': book_info.get('language', ''),
                'Edition': book_info.get('edition', ''),
                'Dimensions': book_info.get('dimensions', ''),
                'MSRP': book_info.get('msrp', ''),
                'Image': book_info.get('image', ''),
                'Status': 'Success'
            })
        else:
            print(f"Error for ISBN {isbn}: {response.status_code}")

            # Write a row with the ISBN, an indication of the error status, and empty fields for other information
            writer.writerow({
                'ISBN': isbn,
                'Status': 'Error',
                'Title': '',
                'Author': '',
                'Publisher': '',
                'Pages': '',
                'Date Published': '',
                'Binding': '',
                'Synopsis': '',
                'Language': '',
                'Edition': '',
                'Dimensions': '',
                'MSRP': '',
                'Image': ''
            })

print("CSV file created successfully.")

Error for ISBN 11-01-1997: 404
Error for ISBN 28036382: 404
Error for ISBN 0741-6555: 404
Error for ISBN 94110: 404
Error for ISBN 882554: 404
Error for ISBN 94188-2554: 404
Error for ISBN 415-642-9993: 404
Error for ISBN 415-642-9995: 404
Error for ISBN 882554: 404
Error for ISBN 94188: 404
Error for ISBN 94110: 404
Error for ISBN 77098: 404
Error for ISBN 404-815: 404
Error for ISBN 04104: 404
Error for ISBN 3005-4: 404
Error for ISBN 3004-6: 404
Error for ISBN 2899-8: 404
Error for ISBN 2898-X: 404
Error for ISBN 2439-9: 404
Error for ISBN 2441-0: 404
Error for ISBN 2440-2: 404
Error for ISBN 425-815: 404
Error for ISBN 604-687-2919: 404
Error for ISBN 617-354-5201: 404
Error for ISBN 94621: 404
Error for ISBN 999-4650: 404
Error for ISBN 632-1281: 404
Error for ISBN 90048: 404
Error for ISBN 213-651-0880: 404
Error for ISBN 95061: 404
Error for ISBN 408-426-9172: 404
Error for ISBN 29681: 404
Error for ISBN 864-862-7833: 404
Error for ISBN 864-862-6944: 404
Error for ISBN 615-793-5

In [15]:
df.head()

,PageNumber,Text,ISBNs
0,1,Feminist Bookstore News\nSource: Reveal Digita...,"[11-01-1997, 28036382]"
1,2,Feminist\nBookstore\nNews EINE\nVolume 20 Numb...,[]
2,3,AUASTE a aA SA Authors\nTHE SECRET LANGUAGE OF...,[]
3,4,Feminist Bookstore News\nAs Wwe go to press......,[]
4,5,Feminist Bookstore News\nyou think — printable...,"[0741-6555, 94110, 882554, 94188-2554]"


In [16]:
import pandas as pd

# Assuming 'ISBNs' is the current column name and 'ISBN' is the desired column name
df.rename(columns={'ISBNs': 'ISBN'}, inplace=True)

In [17]:
page = df[['PageNumber', 'ISBN']]
page.head()

,PageNumber,ISBN
0,1,"[11-01-1997, 28036382]"
1,2,[]
2,3,[]
3,4,[]
4,5,"[0741-6555, 94110, 882554, 94188-2554]"


In [19]:
page['ISBN'] = page['ISBN'].astype(str).str.replace('[\[\]\'\"]', '')
page.head()

C:\Users\salom\AppData\Local\Temp\ipykernel_10800\3003249529.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  page['ISBN'] = page['ISBN'].astype(str).str.replace('[\[\]\'\"]', '')
C:\Users\salom\AppData\Local\Temp\ipykernel_10800\3003249529.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  page['ISBN'] = page['ISBN'].astype(str).str.replace('[\[\]\'\"]', '')


,PageNumber,ISBN
0,1,"11-01-1997, 28036382"
1,2,
2,3,
3,4,
4,5,"0741-6555, 94110, 882554, 94188-2554"


In [22]:
# Split values on comma and explode to create new rows
page['ISBN'] = page['ISBN'].str.split(', ')
page = page.explode('ISBN')

# Display the updated DataFrame
page.head(10)

,PageNumber,ISBN
0,1,11-01-1997
0,1,28036382
1,2,
2,3,
3,4,
4,5,0741-6555
4,5,94110
4,5,882554
4,5,94188-2554
5,6,415-642-9993


In [25]:
import pandas as pd

# Specify the file path
file_path = r'C:\Users\salom\OneDrive\Área de Trabalho\FBN-\data\book_information.csv'

# Read the CSV file into a DataFrame
info = pd.read_csv(file_path)

info

,ISBN,Title,Author,Publisher,Pages,Date Published,Binding,Synopsis,Language,Edition,Dimensions,MSRP,Image,Status
0,0-87286-332-8,Close to the Machine: Technophilia and Its Dis...,"Ullman, Ellen",City Lights Publishers,189.0,1/1/2001,Paperback,"Ullman Tries To Balance Her Life, Close To The...",en,(2nd),"Height: 8 Inches, Length: 5 Inches, Weight: 0....",12.95,https://images.isbndb.com/covers/33/23/9780872...,Success
1,1-880913-16-X,Raising Peaceful Children In A Violent World,"Cecil, Nancy Lee, Roberts, Patricia L.",Publishing/Editing Network,253.0,1997,Paperback,NaN,en,NaN,"Height: 10.25 Inches, Length: 7.25 Inches, Wei...",16.95,https://images.isbndb.com/covers/31/61/9781880...,Success
2,1-55074-113-6,The Science Book for Girls: and Other Intellig...,"Wyatt, Valerie",Kids Can Press,80.0,8/1/1993,Paperback,<p>Studies show that many girls' interest in s...,en,NaN,"Height: 9.5 Inches, Length: 8.5 Inches, Weight...",1.99,https://images.isbndb.com/covers/11/31/9781550...,Success
3,0-670-87367-5,Monkey Bridge,"Cao, Lan",Viking Adult,260.0,7/1/1997,Hardcover,A Memoir Of A Vietnamese Refugee. The Narrator...,en,NaN,"Height: 9.25 Inches, Length: 5.75 Inches, Weig...",23.95,https://images.isbndb.com/covers/36/78/9780670...,Success
4,0-452-27621-7,Storming Heaven's Gate: An Anthology of Spirit...,"Vecchione, Patricia, Sumrall, Amber Coverdale",Plume,496.0,7/1/1997,Paperback,In This Powerful Collection Of Modern Prose An...,en,First Plume Printed Edition,"Height: 8.25 Inches, Length: 5.5 Inches, Weigh...",14.95,https://images.isbndb.com/covers/62/15/9780452...,Success
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1046,1-800-862,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Error
1047,05401-5374,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Error
1048,415-642-9993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Error
1049,415-334-4431,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Error


In [28]:
merge = pd.merge(info, page, on='ISBN', how='left')
merge

,ISBN,Title,Author,Publisher,Pages,Date Published,Binding,Synopsis,Language,Edition,Dimensions,MSRP,Image,Status,PageNumber
0,0-87286-332-8,Close to the Machine: Technophilia and Its Dis...,"Ullman, Ellen",City Lights Publishers,189.0,1/1/2001,Paperback,"Ullman Tries To Balance Her Life, Close To The...",en,(2nd),"Height: 8 Inches, Length: 5 Inches, Weight: 0....",12.95,https://images.isbndb.com/covers/33/23/9780872...,Success,31.0
1,1-880913-16-X,Raising Peaceful Children In A Violent World,"Cecil, Nancy Lee, Roberts, Patricia L.",Publishing/Editing Network,253.0,1997,Paperback,NaN,en,NaN,"Height: 10.25 Inches, Length: 7.25 Inches, Wei...",16.95,https://images.isbndb.com/covers/31/61/9781880...,Success,42.0
2,1-55074-113-6,The Science Book for Girls: and Other Intellig...,"Wyatt, Valerie",Kids Can Press,80.0,8/1/1993,Paperback,<p>Studies show that many girls' interest in s...,en,NaN,"Height: 9.5 Inches, Length: 8.5 Inches, Weight...",1.99,https://images.isbndb.com/covers/11/31/9781550...,Success,42.0
3,0-670-87367-5,Monkey Bridge,"Cao, Lan",Viking Adult,260.0,7/1/1997,Hardcover,A Memoir Of A Vietnamese Refugee. The Narrator...,en,NaN,"Height: 9.25 Inches, Length: 5.75 Inches, Weig...",23.95,https://images.isbndb.com/covers/36/78/9780670...,Success,42.0
4,0-452-27621-7,Storming Heaven's Gate: An Anthology of Spirit...,"Vecchione, Patricia, Sumrall, Amber Coverdale",Plume,496.0,7/1/1997,Paperback,In This Powerful Collection Of Modern Prose An...,en,First Plume Printed Edition,"Height: 8.25 Inches, Length: 5.5 Inches, Weigh...",14.95,https://images.isbndb.com/covers/62/15/9780452...,Success,42.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1192,05401-5374,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Error,139.0
1193,415-642-9993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Error,6.0
1194,415-642-9993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Error,139.0
1195,415-334-4431,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Error,139.0


In [31]:
# Assuming 'merge' is your DataFrame
# Drop rows with NaN in 'Title' and specific ISBN formats
merge = merge[~(merge['Title'].isna() & merge['ISBN'].str.contains(r'^\d{1}-\d{3}-\d{3}-\d{4}$|^\d{3}-\d{3}-\d{4}$|^\d{5}-\d{4}$|^\d{5}$|^\d{6}$'))]

# Display the updated DataFrame
merge

,ISBN,Title,Author,Publisher,Pages,Date Published,Binding,Synopsis,Language,Edition,Dimensions,MSRP,Image,Status,PageNumber
0,0-87286-332-8,Close to the Machine: Technophilia and Its Dis...,"Ullman, Ellen",City Lights Publishers,189.0,1/1/2001,Paperback,"Ullman Tries To Balance Her Life, Close To The...",en,(2nd),"Height: 8 Inches, Length: 5 Inches, Weight: 0....",12.95,https://images.isbndb.com/covers/33/23/9780872...,Success,31.0
1,1-880913-16-X,Raising Peaceful Children In A Violent World,"Cecil, Nancy Lee, Roberts, Patricia L.",Publishing/Editing Network,253.0,1997,Paperback,NaN,en,NaN,"Height: 10.25 Inches, Length: 7.25 Inches, Wei...",16.95,https://images.isbndb.com/covers/31/61/9781880...,Success,42.0
2,1-55074-113-6,The Science Book for Girls: and Other Intellig...,"Wyatt, Valerie",Kids Can Press,80.0,8/1/1993,Paperback,<p>Studies show that many girls' interest in s...,en,NaN,"Height: 9.5 Inches, Length: 8.5 Inches, Weight...",1.99,https://images.isbndb.com/covers/11/31/9781550...,Success,42.0
3,0-670-87367-5,Monkey Bridge,"Cao, Lan",Viking Adult,260.0,7/1/1997,Hardcover,A Memoir Of A Vietnamese Refugee. The Narrator...,en,NaN,"Height: 9.25 Inches, Length: 5.75 Inches, Weig...",23.95,https://images.isbndb.com/covers/36/78/9780670...,Success,42.0
4,0-452-27621-7,Storming Heaven's Gate: An Anthology of Spirit...,"Vecchione, Patricia, Sumrall, Amber Coverdale",Plume,496.0,7/1/1997,Paperback,In This Powerful Collection Of Modern Prose An...,en,First Plume Printed Edition,"Height: 8.25 Inches, Length: 5.5 Inches, Weigh...",14.95,https://images.isbndb.com/covers/62/15/9780452...,Success,42.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1165,0-804-11558-3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Error,138.0
1166,0-553-57715-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Error,138.0
1171,0-553-57489-2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Error,138.0
1178,0-385-48692-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Error,138.0


In [4]:
import pandas as pd
duplicate = pd.read_csv(r"C:\Users\salom\OneDrive\Área de Trabalho\FBN\data\20.4\20.4_booklist.csv")
duplicate

,ISBN,Title,Author,Publisher,Pages,Date Published,Binding,Synopsis,Language,Edition,Dimensions,MSRP,Image,Status,PageNumber
0,0-87286-332-8,Close to the Machine: Technophilia and Its Dis...,"Ullman, Ellen",City Lights Publishers,189.0,1/1/2001,Paperback,"Ullman Tries To Balance Her Life, Close To The...",en,(2nd),"Height: 8 Inches, Length: 5 Inches, Weight: 0....",12.95,https://images.isbndb.com/covers/33/23/9780872...,Success,31.0
1,1-880913-16-X,Raising Peaceful Children In A Violent World,"Cecil, Nancy Lee, Roberts, Patricia L.",Publishing/Editing Network,253.0,1997,Paperback,NaN,en,NaN,"Height: 10.25 Inches, Length: 7.25 Inches, Wei...",16.95,https://images.isbndb.com/covers/31/61/9781880...,Success,42.0
2,1-55074-113-6,The Science Book for Girls: and Other Intellig...,"Wyatt, Valerie",Kids Can Press,80.0,8/1/1993,Paperback,<p>Studies show that many girls' interest in s...,en,NaN,"Height: 9.5 Inches, Length: 8.5 Inches, Weight...",1.99,https://images.isbndb.com/covers/11/31/9781550...,Success,42.0
3,0-670-87367-5,Monkey Bridge,"Cao, Lan",Viking Adult,260.0,7/1/1997,Hardcover,A Memoir Of A Vietnamese Refugee. The Narrator...,en,NaN,"Height: 9.25 Inches, Length: 5.75 Inches, Weig...",23.95,https://images.isbndb.com/covers/36/78/9780670...,Success,42.0
4,0-452-27621-7,Storming Heaven's Gate: An Anthology of Spirit...,"Vecchione, Patricia, Sumrall, Amber Coverdale",Plume,496.0,7/1/1997,Paperback,In This Powerful Collection Of Modern Prose An...,en,First Plume Printed Edition,"Height: 8.25 Inches, Length: 5.5 Inches, Weigh...",14.95,https://images.isbndb.com/covers/62/15/9780452...,Success,42.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
785,0-804-11558-3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Error,138.0
786,0-553-57715-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Error,138.0
787,0-553-57489-2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Error,138.0
788,0-385-48692-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Error,138.0


In [6]:
duplicate = duplicate.drop_duplicates(subset=['ISBN'])
duplicate

,ISBN,Title,Author,Publisher,Pages,Date Published,Binding,Synopsis,Language,Edition,Dimensions,MSRP,Image,Status,PageNumber
0,0-87286-332-8,Close to the Machine: Technophilia and Its Dis...,"Ullman, Ellen",City Lights Publishers,189.0,1/1/2001,Paperback,"Ullman Tries To Balance Her Life, Close To The...",en,(2nd),"Height: 8 Inches, Length: 5 Inches, Weight: 0....",12.95,https://images.isbndb.com/covers/33/23/9780872...,Success,31.0
1,1-880913-16-X,Raising Peaceful Children In A Violent World,"Cecil, Nancy Lee, Roberts, Patricia L.",Publishing/Editing Network,253.0,1997,Paperback,NaN,en,NaN,"Height: 10.25 Inches, Length: 7.25 Inches, Wei...",16.95,https://images.isbndb.com/covers/31/61/9781880...,Success,42.0
2,1-55074-113-6,The Science Book for Girls: and Other Intellig...,"Wyatt, Valerie",Kids Can Press,80.0,8/1/1993,Paperback,<p>Studies show that many girls' interest in s...,en,NaN,"Height: 9.5 Inches, Length: 8.5 Inches, Weight...",1.99,https://images.isbndb.com/covers/11/31/9781550...,Success,42.0
3,0-670-87367-5,Monkey Bridge,"Cao, Lan",Viking Adult,260.0,7/1/1997,Hardcover,A Memoir Of A Vietnamese Refugee. The Narrator...,en,NaN,"Height: 9.25 Inches, Length: 5.75 Inches, Weig...",23.95,https://images.isbndb.com/covers/36/78/9780670...,Success,42.0
4,0-452-27621-7,Storming Heaven's Gate: An Anthology of Spirit...,"Vecchione, Patricia, Sumrall, Amber Coverdale",Plume,496.0,7/1/1997,Paperback,In This Powerful Collection Of Modern Prose An...,en,First Plume Printed Edition,"Height: 8.25 Inches, Length: 5.5 Inches, Weigh...",14.95,https://images.isbndb.com/covers/62/15/9780452...,Success,42.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
785,0-804-11558-3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Error,138.0
786,0-553-57715-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Error,138.0
787,0-553-57489-2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Error,138.0
788,0-385-48692-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Error,138.0


In [8]:
# Assuming 'merge' is your DataFrame
duplicate.to_csv('20.4_booklist.csv', index=False)